# Generate report from exprted photometer data  

Use raw folder with photometer exported data

## Imports

In [ ]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [ ]:
from os import path
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [ ]:
from mkinout import make_input_paths, parse_dir_name

# WORKING_DIR = './reports/export/230712_AAV8-ELISA_sey_GN004240-048'
WORKING_DIR = './reports/export/230801_AAV9-ELISA_sey_GN004240-053'
DATA_DIR = './data'

PARSED_DIR = parse_dir_name(WORKING_DIR)
# WORKLIST_FILE_PATH = path.join(WORKING_DIR,  PARSED_DIR['date'] + '_worklist-ELISA.xls')
# if not path.isfile(WORKLIST_FILE_PATH):
#     raise Exception(f'Invalit worklist file {WORKLIST_FILE_PATH}!')

# PARAMS_FILE_PATH = path.join(WORKING_DIR, PARSED_DIR['date'] + '_Parameters_' + PARSED_DIR['protocol'] + '.csv')
# if not path.isfile(PARAMS_FILE_PATH):
#     raise Exception(f'Invalid parameters file {PARAMS_FILE_PATH}!')

input_files = make_input_paths(WORKING_DIR)
WORKLIST_FILE_PATH = input_files['worklist']
PARAMS_FILE_PATH = input_files['params']

## Layouts

In [ ]:
from readdata import read_layouts

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'


g_lay = read_layouts(path.join(DATA_DIR, PLATE_LAYOUT_ID),
                     path.join(DATA_DIR, PLATE_LAYOUT_NUM),
                     path.join(DATA_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_lay)

## Worklist

In [ ]:
from worklist import read_worklist, check_worklist
from readdata import read_params

g_wl_raw = read_worklist(WORKLIST_FILE_PATH)
g_params = read_params(PARAMS_FILE_PATH)

## Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [ ]:
# TODO: read reference value from parameters
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

from sample import make_concentration
g_reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(g_reference_conc)

## Report generation

In [ ]:
from reportmain import report_plate, check_report_crc
from mkinout import make_output_paths, basename_from_inputdir, parse_dir_name, find_analysis, parse_photometer_filename
from datetime import datetime

def make_input_analysis_txt(work_dir, protocol, plate_id, dt):
    sdt = dt.strftime('%Y%m%d_%H%M%S')
    sd = dt.strftime('%y%m%d')
    f = f'{sd}_{protocol}__{plate_id}_{sdt}.txt'
    p = path.join(work_dir, f)
    if not path.isfile(p):
        raise Exception(f'Invalid plate export file {p}!')
    return p


def gen_report(worklist, params, layout, reference_conc, working_dir):
    reports = []
    alist = find_analysis(working_dir)
    print(alist)
    for analysis_file in alist:
        dc = parse_photometer_filename(analysis_file)
        plate = int(dc['plate'])
        print('Processing plate {} of {}'.format(plate, len(alist)))

        base_name = basename_from_inputdir(WORKING_DIR)
        output_files = make_output_paths(working_dir, base_name, plate)
        report_file_path = output_files['report']

        report_dir = path.dirname(path.abspath(report_file_path))
        info = parse_dir_name(working_dir)
        md, dfres = report_plate(plate, worklist, params, layout,
                    reference_conc, analysis_file, report_dir,
                    info
                    )
        reports.append({'md': md, 'df': dfres, 'path': report_file_path, 'plate': plate})
    return reports

reports = gen_report(g_wl_raw, g_params, g_lay, g_reference_conc,
    WORKING_DIR)

In [ ]:
import reportmdassembly as ra
from reportmdassembly import assembly
from reportmd import save_md
import subprocess

import importlib
importlib.reload(ra)

md_assembly = ra.assembly(reports, protocol=PARSED_DIR['protocol'])
mdfile = '{}_{}.md'.format(PARSED_DIR['date'], PARSED_DIR['protocol'])
md_filepath = path.join(WORKING_DIR, mdfile)
save_md(md_filepath, md_assembly)
docx_path = path.splitext(md_filepath)[0] + '.docx'
print('Generating Word {} for {}'.format(docx_path, md_filepath))
REFERENCE_DOCX = 'C:/work/report-gen/custom-reference.docx'
PANDOC_PATH = 'c:/work/pandoc/pandoc'
subprocess.run([PANDOC_PATH, '-o', docx_path,
                    '-f', 'markdown', '-t', 'docx',
                    '--resource-path', WORKING_DIR,
                    '--reference-doc', REFERENCE_DOCX,
                    md_filepath])

In [ ]:
PDFLATEX_EXE = 'c:/Users/hwn6193/AppData/Local/Programs/MiKTeX/miktex/bin/x64/pdflatex.exe'
REFERENCE_DOCX = 'C:/work/report-gen/custom-reference.docx'
GENERATE_PLATE_PDF = True
GENERATE_PLATE_DOCX = False

for report in reports:
    print('Report for plate {} saved as {}'.format(report['plate'], report['path']))
    save_md(report['path'], report['md'])
    xlsx_file = path.splitext(report['path'])[0] + '_results.xlsx'
    report['df'].to_excel(xlsx_file)

    if GENERATE_PLATE_DOCX:
        report_dir = path.dirname(path.abspath(report['path']))
        docx_path = path.splitext(report['path'])[0] + '.docx'
        print('Generating Word {} for {}'.format(docx_path, report['path']))
        REFERENCE_DOCX = 'C:/work/report-gen/custom-reference.docx'
        PANDOC_PATH = 'c:/work/pandoc/pandoc'
        subprocess.run([PANDOC_PATH, '-o', docx_path,
                            '-f', 'markdown', '-t', 'docx',
                            '--resource-path', report_dir,
                            '--reference-doc', REFERENCE_DOCX,
                            report['path']])

    if GENERATE_PLATE_PDF:
        pdf_path = path.splitext(report['path'])[0] + '.pdf'
        print(f'Generating PDF {pdf_path} for {report["path"]}')
        subprocess.run([PANDOC_PATH, '-o', pdf_path,
                        '--resource-path', report_dir,
                        '--pdf-engine', PDFLATEX_EXE,
                        report['path']])


In [ ]:
CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [3422234280, 3350913816, 931962603]
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        try:
            check_report_crc(report['md'], crc)
        except Exception as e:
            print('{} for {}'.format(e, report['path']))